In [1]:
%%javascript
// builds the TOC
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

# A Decision Tree of Observable Operators

## Part 3: Transformation

> source: http://reactivex.io/documentation/operators.html#tree.  
> (transcribed to RxPY 1.5.7, Py2.7 / 2016-12, Gunther Klessinger, [axiros](http://www.axiros.com))  

**This tree can help you find the ReactiveX Observable operator you’re looking for.**  
See [Part 1](./A Decision Tree of Observable Operators. Part I - Creation.ipynb) for Usage and Output Instructions.  

We also require acquaintance with the [marble diagrams](./Marble Diagrams.ipynb) feature of RxPy.

<h2 id="tocheading">Table of Contents</h2>
<div id="toc"></div>



In [6]:
# Helpers.
# Run this cell always after kernel restarts. All other cells are autonomous.
from __future__ import print_function
import rx
import time
import logging
from random import randint
from rx.testing import marbles

logging.basicConfig(format="%(threadName)s:%(message)s")
log = logging.getLogger("Rx")
log.setLevel(logging.WARNING)

# getting the current thread
import threading
threads = []
def cur_thread():
    def _cur():
        'return a unique number for the current thread'
        n = threading.current_thread().name
        if 'Main' in n:
            return '    M'
        return '%5s' % ('T' + n.rsplit('-', 1)[-1])
    # you could show all running threads via this:
    #threads = ' '.join([t.name for t in threading.enumerate()])
    #return '%s of %s' % (_cur(), threads)
    return _cur()

    
sleep, now = time.sleep, time.time
O = rx.Observable

ts_glob = 0 # global start time
def reset_start_time(show_doc_for=None, title=None, sleep=None):
    'resets global start time and also prints doc strings'
    global ts_glob
    if sleep:
        log('main thread sleeping %ss' % sleep)
        time.sleep(sleep)
    ts_glob, d = time.time(), show_doc_for
    if title:
        header(title)
    if not d:
        return
    # print the function doc if given
    print ('function %s of module %s:\n%s\n%s' % (d.func_name, d.__module__, d.__doc__ or 'n.a.', '-' * 80))
        
rst = reset_start_time

def log(*msg):
    s = ' '.join([str(s) for s in msg])
    print ('%s %s %s' % (dt(ts_glob), cur_thread(), s))

def header(msg):
    print ('\n\n%s %s %s\n' % ('=' * 10, msg, '=' * 10))
    
def rand():
    return randint(0, 100)

def to_int(s):
    return int(s) if s.isdigit() else s

def dt(ts):
    # the time delta of now to given ts (in millis, 1 float)
    return str('%.1f' % ((time.time() - ts) * 1000)).rjust(6)

class ItemGetter:
    'allows to throw an object onto a format string'
    def __init__(self, obj):
        self.obj = obj
    def __getitem__(self, k, d=None):
        return getattr(self.obj, k, d)

class Subscriber:
    def __init__(self, observed_stream, **kw):
        print ('')
        log('New subscription on stream', hash(observed_stream))
        self.ts = time.time() # tstart, for dts at events
        self.name = ' (%(name)s)' % kw if 'name' in kw else ''
        
    def _on(self, what, v=''):
        print ('%s %s [%s] %s: %s%s' % (
                dt(ts_glob), cur_thread(), what, dt(self.ts), v, self.name))

    def on_next     (self, v): return self._on('next', v)
    def on_error    (self, v): return self._on('err ', v)
    def on_completed(self)   : return self._on('cmpl', 'fin')
    
def subs(src, **kw):
    return src.subscribe(Subscriber(src, **kw))

from rx.concurrency import new_thread_scheduler, timeout_scheduler
from rx.testing import marbles

# I want emit the items from an Observable after **transforming** them


## ... one at a time with a function **[map / pluck / pluck_attr](http://reactivex.io/documentation/operators/map.html) **

In [25]:
reset_start_time(O.map, title='map') # alias is "select"
# warming up:
d = subs(O.from_((1, 2 , 3)).map(lambda x: x * 2))



========== map ==========

function select of module rx.linq.observable.select:
Project each element of an observable sequence into a new form
    by incorporating the element's index.

    1 - source.map(lambda value: value * value)
    2 - source.map(lambda value, index: value * value + index)

    Keyword arguments:
    :param Callable[[Any, Any], Any] selector: A transform function to
        apply to each source element; the second parameter of the
        function represents the index of the source element.
    :rtype: Observable

    Returns an observable sequence whose elements are the result of
    invoking the transform function on each element of source.
    
--------------------------------------------------------------------------------

   1.0     M New subscription on stream 276591285
   1.8     M [next]    0.5: 2
   2.4     M [next]    1.1: 4
   2.7     M [next]    1.3: 6
   2.9     M [cmpl]    1.6: fin


In [26]:
rst(O.pluck, title='pluck')
d = subs(O.from_([{'x': 1, 'y': 2}, {'x': 3, 'y': 4}]).pluck('y'))

class Coord:
    def __init__(self, x, y): 
        self.x = x
        self.y = y
rst(title='pluck_attr')        
d = subs(O.from_([Coord(1, 2), Coord(3, 4)]).pluck_attr('y'))




========== pluck ==========

function pluck of module rx.linq.observable.pluck:
Retrieves the value of a specified key using dict-like access (as in
    element[key]) from all elements in the Observable sequence.

    Keyword arguments:
    key {String} The key to pluck.

    Returns a new Observable {Observable} sequence of key values.

    To pluck an attribute of each element, use pluck_attr.

    
--------------------------------------------------------------------------------

   0.9     M New subscription on stream 276588961
   1.4     M [next]    0.5: 2
   1.6     M [next]    0.7: 4
   1.9     M [cmpl]    1.0: fin


========== pluck_attr ==========


   0.5     M New subscription on stream 276588985
   0.9     M [next]    0.3: 2
   1.3     M [next]    0.7: 4
   1.6     M [cmpl]    1.0: fin


## ...by emitting all of the items emitted by corresponding Observables **[flatmap](http://reactivex.io/documentation/operators/flatmap.html) **

In [28]:
rst(O.flat_map)


function select_many of module rx.linq.observable.selectmany:
One of the Following:
    Projects each element of an observable sequence to an observable
    sequence and merges the resulting observable sequences into one
    observable sequence.

    1 - source.select_many(lambda x: Observable.range(0, x))

    Or:
    Projects each element of an observable sequence to an observable
    sequence, invokes the result selector for the source element and each
    of the corresponding inner sequence's elements, and merges the results
    into one observable sequence.

    1 - source.select_many(lambda x: Observable.range(0, x), lambda x, y: x + y)

    Or:
    Projects each element of the source observable sequence to the other
    observable sequence and merges the resulting observable sequences into
    one observable sequence.

    1 - source.select_many(Observable.from_([1,2,3]))

    Keyword arguments:
    selector -- A transform function to apply to each element or an
        observab